# Descripcion del dataset

Diccionario de datos:

Diccionario de datos del dataset principal: db_retail

| Nombre | Nombre de columna | Definicion | Tipo de dato |
|--------|-------------------|------------|--------------|
|ID |ID |Identificador unico de la fila |Int
|Nombre del producto |ProductName |Nombre del producto vendido |String
|Cantidad |ProductQuantity |Cantidad de unidades vendidas |Int
|Precio por unidad |Unit Price | Precio de la unidad vendida |Float
|Pais |Country |Pais en el que se vendio el producto |String
|Categoria |Category |Categoria del producto ordenado |String
|Fecha de orden |OrderDate |Fecha en la que se hizo la orden |DateTime
|Fecha de llegada |ArrivalDate |Fecha en la orden llego al cliente |DateTime
|Venta |Sales |Valor total de la venta |Float 
|Ciudad |City |Ciudad donde se hizo la orden |String

# Elaboracion del taller

Librerias y dependencias necesarias

In [56]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType, IntegerType, DateType #tipos de datos para elaboracion del schema
from pyspark.sql.functions import * #  countDistinct, col, desc, asc, isnan, when, count, avg, min, max

In [2]:
spark = SparkSession.builder.appName("session").getOrCreate() #creando la sesion de spark

## 1. Cargar el dataset y convertirlo a JSON, Parquet y CSV

In [3]:
df = pd.read_excel("./datasets/input/db_retail.xlsx")

In [4]:
df.sample(3)

,ProductName,ProductQuantity,UnitPrice,Country,Category,OrderDate,ArrivalDate,Sales,City,ID
170878,"Sauder Forest Hills Library, Woodland Oak Finish",1,359.499,USA,Furniture,2020-12-26 06:40:15,2021-05-16 07:19:38,359.499,San Francisco,170878
42185,RETROSPOT TEA SET CERAMIC 11 PC,6,4.950,Denmark,Furnishing,2016-10-06 07:56:26,2017-01-11 01:45:55,29.700,Kobenhavn,42185
111120,VINTAGE CHRISTMAS GIFT SACK,4,4.150,Switzerland,Art,2016-02-25 10:37:13,2021-09-24 23:05:49,16.600,Gensve,111120


In [5]:
df.to_csv("./datasets/output/db_retail.csv") #exportar a csv
df.to_parquet("./datasets/output/db_retail.parquet", engine = 'pyarrow') #exportar a parquet
df.to_json("./datasets/output/db_retail.json", orient="records") #exportar a json

## 2. Cargar JSON, Parquet y CSV a PySpark

Leyendo con spark.read.format()

In [6]:
#lectura del CSV
df_spark = spark.read.format("csv")\
                            .option("header", "true")\
                            .option("nullValue", "?")\
                            .option("encoding", "utf8")\
                            .load("./datasets/output/db_retail.csv")
df_spark.show(2)

+---+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|_c0|         ProductName|ProductQuantity|UnitPrice|Country|  Category|          OrderDate|        ArrivalDate|Sales|  City| ID|
+---+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|  0|WHITE HANGING HEA...|              6|     2.55|  Italy|Furnishing|2020-03-01 22:00:44|2021-11-18 00:37:57| 15.3|Torino|  0|
|  1|CREAM CUPID HEART...|              8|     2.75| Sweden|Furnishing|2018-04-20 19:27:37|2021-10-30 13:07:25| 22.0| Boras|  1|
+---+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
only showing top 2 rows



In [7]:
#lectura del parquet
df_spark = spark.read.format("parquet")\
                        .option("header", "true")\
                        .option("nullValue", "?")\
                        .option("encoding", "utf8")\
                        .load("./datasets/output/db_retail.parquet")
df_spark.show(2)

+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|         ProductName|ProductQuantity|UnitPrice|Country|  Category|          OrderDate|        ArrivalDate|Sales|  City| ID|
+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|WHITE HANGING HEA...|              6|     2.55|  Italy|Furnishing|2020-03-01 17:00:44|2021-11-17 19:37:57| 15.3|Torino|  0|
|CREAM CUPID HEART...|              8|     2.75| Sweden|Furnishing|2018-04-20 14:27:37|2021-10-30 08:07:25| 22.0| Boras|  1|
+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
only showing top 2 rows



In [8]:
#lectura del JSON
df_spark = spark.read.format("json")\
                        .option("encoding","utf8")\
                        .load("./datasets/output/db_retail.json")
df_spark.show(2)

+-------------+----------+------+-------+---+-------------+--------------------+---------------+-----+---------+
|  ArrivalDate|  Category|  City|Country| ID|    OrderDate|         ProductName|ProductQuantity|Sales|UnitPrice|
+-------------+----------+------+-------+---+-------------+--------------------+---------------+-----+---------+
|1637195877000|Furnishing|Torino|  Italy|  0|1583100044000|WHITE HANGING HEA...|              6| 15.3|     2.55|
|1635599245000|Furnishing| Boras| Sweden|  1|1524252457000|CREAM CUPID HEART...|              8| 22.0|     2.75|
+-------------+----------+------+-------+---+-------------+--------------------+---------------+-----+---------+
only showing top 2 rows



Leyendo con sprk.read.dtype()

In [9]:
#lectura del CSV
df_spark = spark.read.csv("./datasets/output/db_retail.csv",
                            header = True,
                            sep = ",",
                            nullValue = "?",
                            encoding = "utf8")
df_spark.show(2)

+---+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|_c0|         ProductName|ProductQuantity|UnitPrice|Country|  Category|          OrderDate|        ArrivalDate|Sales|  City| ID|
+---+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|  0|WHITE HANGING HEA...|              6|     2.55|  Italy|Furnishing|2020-03-01 22:00:44|2021-11-18 00:37:57| 15.3|Torino|  0|
|  1|CREAM CUPID HEART...|              8|     2.75| Sweden|Furnishing|2018-04-20 19:27:37|2021-10-30 13:07:25| 22.0| Boras|  1|
+---+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
only showing top 2 rows



In [10]:
#lectura del parquet
df_spark = spark.read.parquet("./datasets/output/db_retail.parquet",
                                header = True,
                                sep = ",",
                                nullValue = "?",
                                encoding = "utf8")
df_spark.show(2)

+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|         ProductName|ProductQuantity|UnitPrice|Country|  Category|          OrderDate|        ArrivalDate|Sales|  City| ID|
+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
|WHITE HANGING HEA...|              6|     2.55|  Italy|Furnishing|2020-03-01 17:00:44|2021-11-17 19:37:57| 15.3|Torino|  0|
|CREAM CUPID HEART...|              8|     2.75| Sweden|Furnishing|2018-04-20 14:27:37|2021-10-30 08:07:25| 22.0| Boras|  1|
+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+------+---+
only showing top 2 rows



In [11]:
#lectura del JSON
df_spark = spark.read.json("./datasets/output/db_retail.json",
                            encoding = "utf8")
df_spark.show(2)

+-------------+----------+------+-------+---+-------------+--------------------+---------------+-----+---------+
|  ArrivalDate|  Category|  City|Country| ID|    OrderDate|         ProductName|ProductQuantity|Sales|UnitPrice|
+-------------+----------+------+-------+---+-------------+--------------------+---------------+-----+---------+
|1637195877000|Furnishing|Torino|  Italy|  0|1583100044000|WHITE HANGING HEA...|              6| 15.3|     2.55|
|1635599245000|Furnishing| Boras| Sweden|  1|1524252457000|CREAM CUPID HEART...|              8| 22.0|     2.75|
+-------------+----------+------+-------+---+-------------+--------------------+---------------+-----+---------+
only showing top 2 rows



## 3. Definir el esquema de las columnas

In [12]:
#funcion que retorna PySpark Schema (lista de schemas de las columnas)
def schema(cols_schema: dict) -> StructType:
    """Define the schema of the dataframe

    Args:
        cols_schema (dict): Dictionary with the columns and its types

    Returns:
        StructType: Schema of the dataframe
    """
    return StructType([StructField(key,
                                cols_schema[key][0],
                                cols_schema[key][1]) for key in cols_schema.keys()])

In [13]:
#crear schema de las columnas con tipos de datos
cols_schema = {
    "ID" : (IntegerType(), False),
    "ProductName" : (StringType(), False),
    "ProductQuantity" : (IntegerType(), False),
    "UnitPrice" : (FloatType(), False),
    "Country" : (StringType(), False),
    "Category" : (StringType(), False),
    "OrderDate" : (DateType(), False),
    "ArrivalDate" : (DateType(), False),
    "Sales" : (FloatType(), False),
    "City" : (StringType(), False)
}

df_schema = schema(cols_schema)


In [14]:
#forma 1 de uso del esquema
df_spark = spark.read.format("parquet")\
                        .option("header", "true")\
                        .option("nullValue", "?")\
                        .option("encoding", "utf8")\
                        .option("schema", df_schema)\
                        .load("./datasets/output/db_retail.parquet")
df_spark.show(3)

+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+-------------+---+
|         ProductName|ProductQuantity|UnitPrice|Country|  Category|          OrderDate|        ArrivalDate|Sales|         City| ID|
+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+-------------+---+
|WHITE HANGING HEA...|              6|     2.55|  Italy|Furnishing|2020-03-01 17:00:44|2021-11-17 19:37:57| 15.3|       Torino|  0|
|CREAM CUPID HEART...|              8|     2.75| Sweden|Furnishing|2018-04-20 14:27:37|2021-10-30 08:07:25| 22.0|        Boras|  1|
|RED WOOLLY HOTTIE...|              6|     3.39|  Italy|Furnishing|2017-06-20 14:43:13|2017-09-07 23:28:04|20.34|Reggio Emilia|  2|
+--------------------+---------------+---------+-------+----------+-------------------+-------------------+-----+-------------+---+
only showing top 3 rows



In [15]:
#forma 2 de uso del esquema
df_spark = spark.read.csv("./datasets/output/db_retail.csv",
                           schema = df_schema,
                           header = True)
df_spark.show(3)

+---+--------------------+---------------+---------+-------+----------+----------+-----------+-----+-------------+
| ID|         ProductName|ProductQuantity|UnitPrice|Country|  Category| OrderDate|ArrivalDate|Sales|         City|
+---+--------------------+---------------+---------+-------+----------+----------+-----------+-----+-------------+
|  0|WHITE HANGING HEA...|              6|     2.55|  Italy|Furnishing|2020-03-01| 2021-11-18| 15.3|       Torino|
|  1|CREAM CUPID HEART...|              8|     2.75| Sweden|Furnishing|2018-04-20| 2021-10-30| 22.0|        Boras|
|  2|RED WOOLLY HOTTIE...|              6|     3.39|  Italy|Furnishing|2017-06-20| 2017-09-08|20.34|Reggio Emilia|
+---+--------------------+---------------+---------+-------+----------+----------+-----------+-----+-------------+
only showing top 3 rows



## 4. Calcular medidas de tendencia central

### PySpark

In [16]:
def get_mean(df) -> dict:
    return {
            'avg_' + col: df.select(
                avg(col).alias('avg_'+col)
            ).collect()[0][0]
            for col in [column for column, type in df.dtypes if type in ['int', 'float']]
        }



def get_mode(df) -> dict:
    dic_moda = {}
    list_dic = []
    for item,_ in df.dtypes:
        key , value = item+'_Moda', df.filter(col(item).isNotNull()).groupBy(item)\
                                                            .count()\
                                                            .orderBy("count", ascending=False)\
                                                            .collect()[0]\
                                                            .__getitem__(item)
        dic_moda[key] = value

    list_dic.append(dic_moda)
    return list_dic



def get_median(df) -> dict:
    return {
        'median_' + col: df.select(
            percentile_approx(col, 0.5).alias('median_'+col)
        ).collect()[0][0]
        for col in [column for column, type in df.dtypes if type in ['int', 'float']]
    }

In [17]:
get_mean(df_spark) #calculo de la media

{'avg_ID': 87799.0,
 'avg_ProductQuantity': 13.251649962636973,
 'avg_UnitPrice': 15.38569570362877,
 'avg_Sales': 36.12007812712544}

In [19]:
get_mode(df_spark) #calculo de la moda

[{'ID_Moda': 148,
  'ProductName_Moda': 'WHITE HANGING HEART T-LIGHT HOLDER',
  'ProductQuantity_Moda': 1,
  'UnitPrice_Moda': 1.649999976158142,
  'Country_Moda': 'USA',
  'Category_Moda': 'Furnishing',
  'OrderDate_Moda': datetime.date(2018, 3, 30),
  'ArrivalDate_Moda': datetime.date(2021, 12, 30),
  'Sales_Moda': 15.0,
  'City_Moda': 'Makati City'}]

In [18]:
get_median(df_spark) #calculo de la mediana

{'median_ID': 87793,
 'median_ProductQuantity': 5,
 'median_UnitPrice': 1.9500000476837158,
 'median_Sales': 13.199999809265137}

### SparkSQL

In [20]:
df_spark.createOrReplaceTempView("df_sql") #creando vista SQL de dataframe

In [21]:
avg_numerical_cols = ", ".join([f'AVG({column}) as AVG_{column}' for column, type in df_spark.dtypes if type in ['int', 'float']]) #obtengo nombre de columnas numericas con funcion de promedio
#obteniendo promedio de columnas numericas
spark.sql(f"""SELECT
        {avg_numerical_cols}
        FROM df_sql""").show()

+-------+-------------------+-----------------+-----------------+
| AVG_ID|AVG_ProductQuantity|    AVG_UnitPrice|        AVG_Sales|
+-------+-------------------+-----------------+-----------------+
|87799.0| 13.251649962636973|15.38569570362877|36.12007812712544|
+-------+-------------------+-----------------+-----------------+



In [22]:
#obteniendo la moda de las columnas
spark.sql(f"""SELECT
        {', '.join([f'MAX({i}) as {i}_Moda' for i,_ in df_spark.dtypes])}
        FROM df_sql""").show()

+-------+--------------------+--------------------+--------------+------------+-------------+--------------+----------------+----------+---------+
|ID_Moda|    ProductName_Moda|ProductQuantity_Moda|UnitPrice_Moda|Country_Moda|Category_Moda|OrderDate_Moda|ArrivalDate_Moda|Sales_Moda|City_Moda|
+-------+--------------------+--------------------+--------------+------------+-------------+--------------+----------------+----------+---------+
| 175598|netTALK DUO VoIP ...|               80995|      22638.48|         USA|          USA|    2021-12-30|      2021-12-30|  168469.6|     Yuma|
+-------+--------------------+--------------------+--------------+------------+-------------+--------------+----------------+----------+---------+



In [23]:
#obteniendo la mediana de las columnas numericas
median_numerical_cols = ', '.join([f"PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY {column}) OVER() AS mediana_{column}" for column, type in df_spark.dtypes if type in ['int', 'float']])
spark.sql(f"""SELECT
        {median_numerical_cols}
        FROM df_sql
        LIMIT 1""").show()

+----------+-----------------------+------------------+------------------+
|mediana_ID|mediana_ProductQuantity| mediana_UnitPrice|     mediana_Sales|
+----------+-----------------------+------------------+------------------+
|   87799.0|                    5.0|1.9500000476837158|13.199999809265137|
+----------+-----------------------+------------------+------------------+



## 5. Calcular medidas de dispersion

In [24]:
def get_std(df) -> dict:
    return {
        'std_' + col: df.select(
            stddev(col).alias('std_'+col)
        ).collect()[0][0]
        for col in [column for column, type in df.dtypes if type in ['int', 'float']]
    }



def get_variance(df) -> dict:
    return {
        'std_' + col: df.select(
            variance(col).alias('std_'+col)
        ).collect()[0][0]
        for col in [column for column, type in df.dtypes if type in ['int', 'float']]
    }
    
 
    
def get_variance_coef(df) -> dict:
    cols_numericas = [column for column, type in df.dtypes if type in ['int', 'float']]
    cv = [(stddev(col)/avg(col)).alias('cv_' + col) for col in cols_numericas]
    results = df_spark.select(cv).first()
    coef_var_dict = dict()

    for col in cols_numericas:
        cvi= results['cv_'+ col]
        coef_var_dict[col] = cvi
    return coef_var_dict

In [25]:
#obteniendo la desviacion estandar
get_std(df_spark)

{'std_ID': 50691.209297076355,
 'std_ProductQuantity': 278.00679898616784,
 'std_UnitPrice': 156.94964092652415,
 'std_Sales': 594.8228326126351}

In [26]:
#obteniendo la varianza
get_variance(df_spark)

{'std_ID': 2569598700.0,
 'std_ProductQuantity': 77287.78028253552,
 'std_UnitPrice': 24633.189786964864,
 'std_Sales': 353814.2021973189}

In [27]:
#obteniendo el coeficiente de variacion
get_variance_coef(df_spark)

{'ID': 0.5773552010509955,
 'ProductQuantity': 20.97903278233337,
 'UnitPrice': 10.201010337771534,
 'Sales': 16.467927630697325}

## 6. Realizar GroupBys con PySpark, SparkSQL y Pandas

### PySpark

In [28]:
#calculo de las ventas por pais
df_spark.groupBy('Country').sum('Sales').show(3)

+-----------+------------------+
|    Country|        sum(Sales)|
+-----------+------------------+
|     Sweden|195836.90933071822|
|Philippines|199859.00949585438|
|  Singapore| 194709.9496653825|
+-----------+------------------+
only showing top 3 rows



In [29]:
#calculo de las ventas por ciudad
df_spark.groupBy('City').sum('Sales').show(3)

+---------+-----------------+
|     City|       sum(Sales)|
+---------+-----------------+
|Minato-ku|100216.9497204572|
| Salzburg|98107.91977241635|
|   Madrid|70867.61998099089|
+---------+-----------------+
only showing top 3 rows



In [30]:
#calculo de ventas por cantidad de productos ordenados
df_spark.groupBy('ProductQuantity').sum('Sales').show(3)

+---------------+------------------+
|ProductQuantity|        sum(Sales)|
+---------------+------------------+
|             31|126.78999948501587|
|             53|  540.069995880127|
|            108| 5260.679988861084|
+---------------+------------------+
only showing top 3 rows



In [31]:
#calculo de unidades ordenadas por pais
df_spark.groupBy('Country').sum('ProductQuantity').show(3)

+-----------+--------------------+
|    Country|sum(ProductQuantity)|
+-----------+--------------------+
|     Sweden|              112326|
|Philippines|              116096|
|  Singapore|              111412|
+-----------+--------------------+
only showing top 3 rows



In [32]:
#calculo de unidades ordenadas por ciudad
df_spark.groupBy('City').sum('ProductQuantity').show(3)

+---------+--------------------+
|     City|sum(ProductQuantity)|
+---------+--------------------+
|Minato-ku|               55538|
| Salzburg|               53656|
|   Madrid|               40485|
+---------+--------------------+
only showing top 3 rows



### SparkSQL

In [33]:
#calculo de las ventas por pais
spark.sql("""
    SELECT  Country, sum(Sales) as SalesByCountry
    FROM df_sql
    GROUP BY Country
""").show(3)

+-----------+------------------+
|    Country|    SalesByCountry|
+-----------+------------------+
|     Sweden|195836.90933071822|
|Philippines|199859.00949585438|
|  Singapore| 194709.9496653825|
+-----------+------------------+
only showing top 3 rows



In [34]:
#calculo de las ventas por ciudad
spark.sql("""
    SELECT  City, sum(Sales) as SalesByCity
    FROM df_sql
    GROUP BY City
""").show(3)

+---------+-----------------+
|     City|      SalesByCity|
+---------+-----------------+
|Minato-ku|100216.9497204572|
| Salzburg|98107.91977241635|
|   Madrid|70867.61998099089|
+---------+-----------------+
only showing top 3 rows



In [35]:
#calculo de ventas por cantidad de productos ordenados
spark.sql("""
    SELECT  ProductQuantity, sum(Sales) AS SalesByProductQuantity
    FROM df_sql
    GROUP BY ProductQuantity
""").show(3)

+---------------+----------------------+
|ProductQuantity|SalesByProductQuantity|
+---------------+----------------------+
|             31|    126.78999948501587|
|             53|      540.069995880127|
|            108|     5260.679988861084|
+---------------+----------------------+
only showing top 3 rows



In [36]:
#calculo de unidades ordenadas por pais
spark.sql("""
    SELECT  Country, sum(ProductQuantity) AS ProductQuantityOrderedByCountry
    FROM df_sql
    GROUP BY Country
""").show(3)

+-----------+-------------------------------+
|    Country|ProductQuantityOrderedByCountry|
+-----------+-------------------------------+
|     Sweden|                         112326|
|Philippines|                         116096|
|  Singapore|                         111412|
+-----------+-------------------------------+
only showing top 3 rows



In [37]:
#calculo de unidades ordenadas por ciudad
spark.sql("""
    SELECT  City, sum(ProductQuantity) AS ProductQuantityOrderedByCity
    FROM df_sql
    GROUP BY City
""").show(3)

+---------+----------------------------+
|     City|ProductQuantityOrderedByCity|
+---------+----------------------------+
|Minato-ku|                       55538|
| Salzburg|                       53656|
|   Madrid|                       40485|
+---------+----------------------------+
only showing top 3 rows



### Pandas

In [38]:
#calculo de cantidad de unidades que llegan al destino segun la fecha
df.groupby(["ArrivalDate"]).agg({'ID':'sum'})

,ID
ArrivalDate,
2016-01-09 19:10:02,102061
2016-01-15 13:03:24,85610
2016-01-19 10:32:55,84821
2016-01-19 13:31:19,9468
2016-01-20 05:33:41,18692
...,...
2021-12-30 23:54:14,134689
2021-12-30 23:56:05,3375
2021-12-30 23:57:14,14703


## 7. Inner, left y right join con PySpark, SparkSQL y Pandas

### PySpark

In [60]:
df_cars = pd.read_excel("./datasets/input/db_cars.xlsx") 
df_cars.to_parquet("./datasets/output/db_cars.parquet", engine="pyarrow") #paso excel a parquet

In [62]:
df_cars = spark.read.parquet("./datasets/output/db_cars.parquet", header = True) #lectura de dataset
df_cars.show(2)

+---------------+---------+------+-------------------+-----------+-----+-------+-------------------+-------------------+---+
|ProductQuantity|UnitPrice| Sales|          OrderDate|   Category| City|Country|        ArrivalDate|        ProductName| ID|
+---------------+---------+------+-------------------+-----------+-----+-------+-------------------+-------------------+---+
|             30|     95.7|2871.0|2017-02-07 17:47:58|Motorcycles|  NYC|    USA|2019-08-20 04:58:17|      Honda Dio Dlx|  0|
|             34|    81.35|2765.9|2018-03-02 18:13:56|Motorcycles|Reims| France|2021-06-24 23:55:12|Honda Eco Deluxe Es|  1|
+---------------+---------+------+-------------------+-----------+-----+-------+-------------------+-------------------+---+
only showing top 2 rows



In [71]:
#inner jpoin para saber que ordenes tienen tanto productos de retail como carros
df_spark.join(df_cars, "ID", "inner")\
        .select(df_spark.ID, df_spark.ProductName, df_cars.ProductName, (df_spark.Sales + df_cars.Sales), (df_spark.ProductQuantity + df_cars.ProductQuantity))\
        .show(5)

+---+--------------------+-------------------+------------------+-----------------------------------+
| ID|         ProductName|        ProductName|   (Sales + Sales)|(ProductQuantity + ProductQuantity)|
+---+--------------------+-------------------+------------------+-----------------------------------+
|  0|WHITE HANGING HEA...|      Honda Dio Dlx| 2886.300000190735|                                 36|
|  1|CREAM CUPID HEART...|Honda Eco Deluxe Es|            2787.9|                                 42|
|  2|RED WOOLLY HOTTIE...|Honda Eco Deluxe Es| 3904.680000152588|                                 47|
|  3|SET 7 BABUSHKA NE...|      Bmw R 1200 Gs|3762.0000001907347|                                 47|
|  4|BOX OF VINTAGE JI...|      Bmw R 1200 Gs|  5220.12000038147|                                 52|
+---+--------------------+-------------------+------------------+-----------------------------------+
only showing top 5 rows



In [85]:
#left join para obtener todas las ventas de vehiculos y aquellas ordenes de retail asociadas a estas
df_cars.join(df_spark, "ID", "left")\
       .select(df_cars.ID, df_cars.ProductName, df_cars.ProductQuantity ,df_spark.ProductName, df_spark.ProductQuantity, 
       (df_cars.ProductQuantity + df_spark.ProductQuantity))\
       .show(3) #df_cars es el conjunto de la izquierda, df_spark el de la derecha

In [87]:
#right join para obtener todas las ordenes de retail y las ventas de vehiculos asociadas a estas
df_spark.join(df_cars, "ID", "right")\
        .select(df_spark.ID, df_spark.ProductName, df_spark.ProductQuantity ,df_cars.ProductName, df_cars.ProductQuantity, 
        (df_spark.ProductQuantity + df_cars.ProductQuantity))\
        .show(3)

+---+--------------------+---------------+-------------------+---------------+-----------------------------------+
| ID|         ProductName|ProductQuantity|        ProductName|ProductQuantity|(ProductQuantity + ProductQuantity)|
+---+--------------------+---------------+-------------------+---------------+-----------------------------------+
|  0|WHITE HANGING HEA...|              6|      Honda Dio Dlx|             30|                                 36|
|  1|CREAM CUPID HEART...|              8|Honda Eco Deluxe Es|             34|                                 42|
|  2|RED WOOLLY HOTTIE...|              6|Honda Eco Deluxe Es|             41|                                 47|
+---+--------------------+---------------+-------------------+---------------+-----------------------------------+
only showing top 3 rows



### SparkSQL

In [88]:
df_cars.createOrReplaceTempView("df_cars_sql") #creando vista SQL de dataframe de ventas carros

In [93]:
#inner join para saber que ordenes tienen tanto productos de retail como carros
spark.sql("""
          SELECT Retail.ID, Retail.ProductName AS RetailProduct, Cars.ProductName AS CarsProduct, 
          (Retail.Sales + Cars.Sales) AS TotalSales, (Retail.ProductQuantity + Cars.ProductQuantity) AS TotalUnits
          FROM df_sql AS Retail
          INNER JOIN df_cars_sql AS Cars
          ON Retail.ID = Cars.ID
          """).show(3)

+---+--------------------+-------------------+-----------------+----------+
| ID|       RetailProduct|        CarsProduct|       TotalSales|TotalUnits|
+---+--------------------+-------------------+-----------------+----------+
|  0|WHITE HANGING HEA...|      Honda Dio Dlx|2886.300000190735|        36|
|  1|CREAM CUPID HEART...|Honda Eco Deluxe Es|           2787.9|        42|
|  2|RED WOOLLY HOTTIE...|Honda Eco Deluxe Es|3904.680000152588|        47|
+---+--------------------+-------------------+-----------------+----------+
only showing top 3 rows



In [95]:
#left join para obtener todas las ventas de vehiculos y aquellas ordenes de retail asociadas a estas
spark.sql("""
          SELECT Cars.ID, Cars.ProductName AS CarsProduct, Cars.ProductQuantity AS CarsQuanity, 
          Retail.ProductName AS RetailProduct, Retail.ProductQuantity AS RetailQuanity, 
          (Retail.ProductQuantity + Cars.ProductQuantity) AS TotalUnits, (Retail.Sales + Cars.Sales) AS TotalSales
          FROM df_cars_sql AS Cars
          LEFT JOIN df_sql AS Retail
          ON Cars.ID = Retail.ID
          """).show(3)

+---+-------------------+-----------+--------------------+-------------+----------+-----------------+
| ID|        CarsProduct|CarsQuanity|       RetailProduct|RetailQuanity|TotalUnits|       TotalSales|
+---+-------------------+-----------+--------------------+-------------+----------+-----------------+
|  0|      Honda Dio Dlx|         30|WHITE HANGING HEA...|            6|        36|2886.300000190735|
|  1|Honda Eco Deluxe Es|         34|CREAM CUPID HEART...|            8|        42|           2787.9|
|  2|Honda Eco Deluxe Es|         41|RED WOOLLY HOTTIE...|            6|        47|3904.680000152588|
+---+-------------------+-----------+--------------------+-------------+----------+-----------------+
only showing top 3 rows



In [96]:
#right join para obtener todas las ordenes de retail y las ventas de vehiculos asociadas a estas
spark.sql("""
          SELECT Retail.ID, Retail.ProductName AS RetailProduct, Retail.ProductQuantity AS RetailQuanity, 
          Cars.ProductName AS CarsProduct, Cars.ProductQuantity AS CarsQuanity,
          (Retail.Sales + Cars.Sales) AS TotalSales, (Retail.ProductQuantity + Cars.ProductQuantity) AS TotalUnits
          FROM df_sql AS Retail
          RIGHT JOIN df_cars_sql AS Cars
          ON Retail.ID = Cars.ID
          """).show(3)

+---+--------------------+-------------+-------------------+-----------+-----------------+----------+
| ID|       RetailProduct|RetailQuanity|        CarsProduct|CarsQuanity|       TotalSales|TotalUnits|
+---+--------------------+-------------+-------------------+-----------+-----------------+----------+
|  0|WHITE HANGING HEA...|            6|      Honda Dio Dlx|         30|2886.300000190735|        36|
|  1|CREAM CUPID HEART...|            8|Honda Eco Deluxe Es|         34|           2787.9|        42|
|  2|RED WOOLLY HOTTIE...|            6|Honda Eco Deluxe Es|         41|3904.680000152588|        47|
+---+--------------------+-------------+-------------------+-----------+-----------------+----------+
only showing top 3 rows



### Pandas

In [97]:
#leyendo datasets con pandas
df_retail = pd.read_excel("./datasets/input/db_retail.xlsx")
df_cars = pd.read_excel("./datasets/input/db_cars.xlsx")

In [99]:
#inner join para saber que ordenes tienen tanto productos de retail como carros
pd.merge(df_retail, df_cars, on = "ID", how = "inner")\
  .sample(3)

,ProductName_x,ProductQuantity_x,UnitPrice_x,Country_x,Category_x,OrderDate_x,ArrivalDate_x,Sales_x,City_x,ID,ProductQuantity_y,UnitPrice_y,Sales_y,OrderDate_y,Category_y,City_y,Country_y,ArrivalDate_y,ProductName_y
2021,CHARLOTTE BAG PINK POLKADOT,1,0.85,France,Accessories,2019-01-02 23:00:14,2020-08-08 08:02:21,0.85,Strasbourg,2021,41,68.24,2797.84,2021-08-29 03:08:43,Planes,Reims,France,2021-10-04 01:06:44,avianca B747
312,JUMBO BAG BAROQUE BLACK WHITE,1,1.95,Spain,Accessories,2019-09-26 23:03:12,2021-06-30 08:31:37,1.95,Sevilla,312,34,96.73,3288.82,2017-07-01 12:01:23,Classic Cars,Espoo,Finland,2020-10-26 15:25:06,Volkswagen Jetta Trendline Clasico 2000cc At Aa
2541,PLASTERS IN TIN VINTAGE PAISLEY,12,1.65,Denmark,Art,2019-01-21 00:47:11,2021-10-28 00:59:38,19.80,Aaarhus,2541,24,99.57,2389.68,2018-02-20 09:26:54,Ships,London,UK,2020-05-15 15:37:19,44 cuddy cabin series


In [101]:
#left join para obtener todas las ventas de vehiculos y aquellas ordenes de retail asociadas a estas
pd.merge(df_cars, df_retail, on = "ID", how = "left")\
.sample(3)

,ProductQuantity_x,UnitPrice_x,Sales_x,OrderDate_x,Category_x,City_x,Country_x,ArrivalDate_x,ProductName_x,ID,ProductName_y,ProductQuantity_y,UnitPrice_y,Country_y,Category_y,OrderDate_y,ArrivalDate_y,Sales_y,City_y
2408,46,69.12,3179.52,2018-04-24 11:52:13,Trains,NYC,USA,2019-02-25 02:28:24,Mehano T111,2408,SWEETHEART CERAMIC TRINKET BOX,4,1.25,Switzerland,Furnishing,2018-05-25 15:54:27,2021-06-01 12:18:15,5.00,Gensve
1578,30,79.98,2399.40,2020-03-15 15:34:13,Motorcycles,Nantes,France,2021-05-05 21:54:45,Honda Dio Dlx,1578,RED RETROSPOT PEG BAG,6,2.10,Denmark,Accessories,2021-09-03 01:14:53,2021-12-21 20:04:13,12.60,Kobenhavn
1695,21,73.60,1545.60,2017-07-14 04:27:24,Classic Cars,North Sydney,Australia,2020-11-07 23:48:24,Ford F-100 Ranger Lariat,1695,HANGING CHICK YELLOW DECORATION,1,1.45,Sweden,Furnishing,2018-07-28 04:02:41,2020-06-14 10:54:45,1.45,Lule


In [102]:
#right join para obtener todas las ordenes de retail y las ventas de vehiculos asociadas a estas
pd.merge(df_retail, df_cars, on = "ID", how = "right")\
  .sample(3)

,ProductName_x,ProductQuantity_x,UnitPrice_x,Country_x,Category_x,OrderDate_x,ArrivalDate_x,Sales_x,City_x,ID,ProductQuantity_y,UnitPrice_y,Sales_y,OrderDate_y,Category_y,City_y,Country_y,ArrivalDate_y,ProductName_y
2090,CAKE STAND VICTORIAN FILIGREE LARGE,1,8.50,Australia,Appliances,2016-11-13 15:47:08,2020-01-31 05:45:01,8.50,North Sydney,2090,44,100.00,4884.88,2021-02-03 18:21:40,Vintage Cars,San Rafael,USA,2021-08-21 08:47:52,Volkwagen Beetle
1291,BAKING SET SPACEBOY DESIGN,1,4.95,Sweden,Furnishing,2019-01-23 16:29:17,2019-06-18 01:02:51,4.95,Lule,1291,43,92.16,3962.88,2018-08-30 18:31:58,Vintage Cars,Madrid,Spain,2019-12-23 07:28:36,1968-1969 Dodge Charger
55,CERAMIC STRAWBERRY CAKE MONEY BANK,4,1.45,Philippines,Appliances,2016-05-19 14:00:08,2017-11-23 06:40:04,5.80,Makati City,55,29,96.34,2793.86,2021-03-30 07:26:30,Motorcycles,Melbourne,Australia,2021-05-25 13:28:29,Bajaj Pulsar Ns 200


## 8. Distinct Count de columnas categoricas

In [46]:
for col in [column for column, type in df_spark.dtypes if type in ['string']]:
    df_spark.groupBy(col)\
    .agg(count(col)\
    .alias('Cantidad'),round((count(col)/df_spark.count())*100,2)\
    .alias('Share')).show()

+--------------------+--------+-----+
|         ProductName|Cantidad|Share|
+--------------------+--------+-----+
|SET/10 BLUE POLKA...|     235| 0.13|
|POTTING SHED SOW ...|       1|  0.0|
|SET/3 RED GINGHAM...|     336| 0.19|
|MAGNETS PACK OF 4...|      87| 0.05|
|PINK  HONEYCOMB P...|      46| 0.03|
|PAPERWEIGHT KINGS...|      17| 0.01|
|GLASS CAKE COVER ...|       2|  0.0|
|JUMBO SHOPPER VIN...|     814| 0.46|
|SET OF 4 ENGLISH ...|     142| 0.08|
|VINTAGE PAISLEY S...|     216| 0.12|
|SET OF 4 NAPKIN C...|     114| 0.06|
|RED FLOCK LOVE HE...|      58| 0.03|
|SET OF 2 ROUND TI...|      67| 0.04|
|SET/10 PINK POLKA...|     318| 0.18|
|ZINC METAL HEART ...|     493| 0.28|
|WOODEN ROUNDERS G...|     311| 0.18|
|CREAM SWEETHEART ...|     481| 0.27|
|12 RED ROSE PEG P...|      86| 0.05|
|  GREEN ROSE WASHBAG|      49| 0.03|
|SET OF 3 CAKE TIN...|    1232|  0.7|
+--------------------+--------+-----+
only showing top 20 rows

+-----------+--------+-----+
|    Country|Cantidad|Share|
+---

## 9. Conclusiones medidas de tendencia central y dispersion

1. En promedio el precio de una unidad es de $15.38 USD, asi mismo el precio maximo promedio esperado por unidad es de unos $172,33 USD (mean + std) 

2. En promedio las ordenes son de 13 productos, asi mismo el maximo de productos ordenados esperado es de 291 prductos

3. El pais que mas aporta ventas en cuanto a volumen, productos adquiridos, es Estados Unidos.  

4. A la hora de realizar un pedido los cliente compran en su mayoria un producto (moda ProductQuantity), esto concuerda con la moda vista en ventas totales y con el promedio del precio por unidad, ya que el total de ventas mas percibido en el conjunto de datos es de $15 USD y asu vez, el precio promedio de la unidad esta en unos $15 USD.  

5. El producto mas vendido es WHITE HANGING T-LIGHT HOLDER (moda de ProductName) de la categoria Furnishing (moda de Category) y la ciudad en la que mas ordenes se han registrado es Makati City (moda de City).